In [ ]:
# Image Processing: DCT, FFT, Noise, and Filtering Analysis
# (Generated by ChatGPT)

from __future__ import annotations
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import fftpack
from scipy.fft import fft2, ifft2, fftshift
from scipy.signal import wiener
from scipy.ndimage import uniform_filter, median_filter, gaussian_filter
from skimage import data, img_as_float, util
from skimage.color import rgb2gray
from skimage.util import random_noise
import imageio
import warnings

warnings.filterwarnings('ignore')

IMAGE_PATH = None
SAVE_RESULTS = False
OUTPUT_DIR = 'results'
GAUSS_VAR = 0.01
SP_AMOUNT = 0.05
SPECKLE_VAR = 0.2
MASK_SIZES = [3, 5, 7, 9, 11]

def ensure_outdir(path: str):
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)

def load_image(path: str | None):
    if path:
        im = imageio.v2.imread(path)
        if im.ndim == 3:
            im = rgb2gray(im)
        return img_as_float(im)
    return img_as_float(data.camera())

def dct2(a: np.ndarray) -> np.ndarray:
    return fftpack.dct(fftpack.dct(a.T, norm='ortho').T, norm='ortho')

def idct2(a: np.ndarray) -> np.ndarray:
    return fftpack.idct(fftpack.idct(a.T, norm='ortho').T, norm='ortho')

def pearson_corr(a: np.ndarray, b: np.ndarray) -> float:
    a_f = a.ravel()
    b_f = b.ravel()
    a_m = a_f.mean()
    b_m = b_f.mean()
    num = np.sum((a_f - a_m) * (b_f - b_m))
    den = np.sqrt(np.sum((a_f - a_m) ** 2) * np.sum((b_f - b_m) ** 2))
    return float(num / den) if den != 0 else 0.0

def radial_profile(data: np.ndarray) -> np.ndarray:
    y, x = np.indices(data.shape)
    center = np.array(data.shape) // 2
    r = np.hypot(x - center[1], y - center[0]).astype(np.int32)
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / (nr + 1e-12)
    return radialprofile

def apply_filters_for_sizes(noisy_img: np.ndarray, sizes: list[int]) -> dict:
    results = {'mean': {}, 'gaussian': {}, 'median': {}}
    for s in sizes:
        results['mean'][s] = uniform_filter(noisy_img, size=s)
        sigma = max(0.3, s / 6.0)
        results['gaussian'][s] = gaussian_filter(noisy_img, sigma=sigma)
        median_uint8 = median_filter((noisy_img * 255).astype(np.uint8), size=s)
        results['median'][s] = median_uint8.astype(np.float32) / 255.0
    return results

def evaluate_filters(original: np.ndarray, filters_dict: dict, sizes: list[int]) -> dict:
    corr_vs_size = {name: [] for name in filters_dict.keys()}
    for name, d in filters_dict.items():
        for s in sizes:
            filtered = d[s]
            corr = pearson_corr(original, np.clip(filtered, 0, 1))
            corr_vs_size[name].append(corr)
    return corr_vs_size

def main():
    img = load_image(IMAGE_PATH)
    h, w = img.shape
    print(f'Image loaded: shape={img.shape}')

    plt.figure(figsize=(5, 5))
    plt.title('Original image')
    plt.axis('off')
    plt.imshow(img, cmap='gray')
    plt.show()

    dct_coeffs = dct2(img)
    recon_dct = idct2(dct_coeffs)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.title('DCT coefficients (log magnitude)')
    plt.axis('off')
    plt.imshow(np.log1p(np.abs(dct_coeffs)), cmap='gray')
    plt.subplot(1, 3, 2)
    plt.title('Reconstructed from IDCT')
    plt.axis('off')
    plt.imshow(np.clip(recon_dct, 0, 1), cmap='gray')
    plt.subplot(1, 3, 3)
    plt.title('Difference (orig - idct)')
    plt.axis('off')
    plt.imshow(img - np.clip(recon_dct, 0, 1), cmap='bwr')
    plt.colorbar(fraction=0.046, pad=0.01)
    plt.show()

    fft_coeffs = fft2(img)
    fft_shift = fftshift(fft_coeffs)
    magnitude_spectrum = np.log1p(np.abs(fft_shift))
    recon_fft = np.real(ifft2(fft_coeffs))

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.title('FFT magnitude spectrum (log)')
    plt.axis('off')
    plt.imshow(magnitude_spectrum, cmap='gray')
    plt.subplot(1, 3, 2)
    plt.title('Reconstructed from iFFT')
    plt.axis('off')
    plt.imshow(np.clip(recon_fft, 0, 1), cmap='gray')
    plt.subplot(1, 3, 3)
    plt.title('Difference (orig - ifft)')
    plt.axis('off')
    plt.imshow(img - np.clip(recon_fft, 0, 1), cmap='bwr')
    plt.colorbar(fraction=0.046, pad=0.01)
    plt.show()

    corr_idct = pearson_corr(img, np.clip(recon_dct, 0, 1))
    corr_ifft = pearson_corr(img, np.clip(recon_fft, 0, 1))
    print(f'Pearson correlation original vs IDCT reconstruction: {corr_idct:.6f}')
    print(f'Pearson correlation original vs IFFT reconstruction: {corr_ifft:.6f}')
    closer = 'IDCT' if corr_idct > corr_ifft else 'IFFT'
    print(f'Closer to original by Pearson correlation: {closer}')

if __name__ == '__main__':
    main()
